In [38]:
import numpy as np 
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import numpy.linalg as lng 
import time
from scipy.stats import norm
rom scipy.sparse import csr_matrix as sparse
from scipy.sparse.linalg import spsolve

# paramètres et fonctions

In [21]:
K=100
X_min=0
X_max=200
T=1
sigma=0.2
r=0.1


def t(n):
    return n*D_t
def s(j,h):
    return X_min+j*h

def alpha(j,h):
    var=(sigma*s(j,h))/h
    return 0.5*var*var
                
def beta(j,h):
    return 0.5*r*s(j,h)*(1/h)
                
def uleft(t_n):
    return K*np.exp(-r*t_n)-X_min
def uright(t_n):
    return 0
def phi(s_j,dim):
       return np.maximum(K-s_j,0).reshape(dim,1)
    
def q(t,dim,h):
    y=np.zeros((dim,1))
    y[0]=(-alpha(1,h)+beta(1,h))*uleft(t)
    y[-1]=(-alpha(1,h)+beta(1,h))*uright(t)
    return y 

#Black and Scholes formula
def BS_PUT(S, K, T, r, sigma):
    N1 = norm.cdf
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    return K*np.exp(-r*T)*N1(-d2) - S*N1(-d1)


# function calcul tableau

## Calcul U,B, CLF, U(s_barre), Erreur et temps de calcul

In [41]:
def final_func(I,N,Sc):
    tp1=time.time()
    #initialisation des param
    h=(X_max-X_min)/(I+1)
    D_t=T/N
    SCHEMA=Sc
    S_1=X_min +h*np.arange(1,I+1)
    
    #création de Q
    Q=np.zeros((I,I))
    
    for i in range(I):
        for j in range(I):
            if (i==j):
                Q[i,j]=2*alpha(i+1,h)+r
            elif(i==j-1 ):
                Q[i,j]=-alpha(i+1,h)-beta(i+1,h)
            elif(i==j+1 ):
                Q[i,j]=-alpha(i+1,h)+beta(i+1,h)
                
                
    #Q = sparse((2 * alpha(np.arange(1, I+1), h) + r, np.arange(I), np.arange(I)), shape=(I, I))
    #Q += sparse((-alpha(np.arange(2, I+1), h) - beta(np.arange(2, I+1), h), np.arange(1, I), np.arange(I-1)), shape=(I, I))
    #Q += sparse((-alpha(np.arange(1, I), h) + beta(np.arange(1, I), h), np.arange(I-1), np.arange(1, I)), shape=(I, I))

                
    #init de U
    U=phi(S_1,I)
    U_0=U
    Id=np.identity(I)
    
    #matrice d'amplification
    B=Id-D_t*Q
    
    #nombre CFL  
    mu= (D_t*pow(sigma,2)*pow(X_max,2))*(1/pow(h,2))
    
    #Calcul de U selon le schema
    for n in range(0,N+1):

        t=n*D_t
        if (SCHEMA=='IE'):
            U=lng.solve(Id+D_t*Q,U - D_t*q((n+1)*D_t,I,h))
        if (SCHEMA=='EE'):
            U=(Id-D_t*Q)@U - D_t*q(t,I,h)   
        if(SCHEMA=='CN'):
            U = lng.solve(Id+D_t*0.5*Q,(Id-0.5*D_t*Q)@U-0.5*D_t*q(t,I,h)-0.5*D_t*q((n+1)*D_t,I,h))  
            
    
    #calcul de U(s_barre) et l'erreur
    Sval=90 
    i=0
    while (S_1[i]<Sval):
        i+=1

    U_sval=float(((S_1[i]-Sval)/h)*U[i-1] + ((Sval-S_1[i-1])/h)*U[i])
    V = BS_PUT(Sval,K,T,r,sigma)
    erreur = abs(U_sval - V)
    tp2=time.time()
    temps_calc=tp2-tp1
    return [U_sval,erreur,B,mu,temps_calc]  


    
    

0.003376007080078125


## Norme matrice d'amplification et nombre CLF 

### Function affichage results ( B , normes de B et CLF number ) 

In [23]:
def norm_ampl_matrix_CLF(B,CLF):
    norm_inf=lng.norm(B,np.inf)
    norm_2=lng.norm(B,2)
    print('Amplification ''Matrix : \n\n',B,
          '\n \n',
          'Norme 2 =',norm_2,'  Norme inf =',norm_inf, '  CLF Number =',CLF)

## Function calcul tabelaux complets selon scheme (I,N,U(s_barre), Erreur, ordre) 

In [28]:
def mother_of_all_func(schema_choix):
    i=1
    j=i*10
    df_1=pd.DataFrame()
    df_2=pd.DataFrame()
    inter=[0,1]
    while i<7: 
        if(schema_choix=='EE'):
            m=pow(j,2)/10
            inter[0]='I=N'
            inter[1]='N=(I**2)/10'
            if(i==6): break
        else:
            inter[0]='I=N'
            inter[1]='N=I/10'
            m=j/10
        j=int(j)
        m=int(m)
        k=j
        tab1=final_func(j,k,schema_choix) #Results for I=N
        tab2=final_func(j,m,schema_choix) #Results for (I**2/10 si EE sinon I/10)

        df_1.at[i,'I']=j
        df_1.at[i,'N']=k
        df_1.at[i,'U_barre']=tab1[0]
        df_1.at[i,'Error']=tab1[1]

        df_2.at[i,'I']=j
        df_2.at[i,'N']=m
        df_2.at[i,'U_barre']=tab2[0]
        df_2.at[i,'Error']=tab2[1]
        
        j=j*2 
        i=i+1

    #calcul de l'ordre
    for i in range(1,df_1.shape[0]+1):

        if i>1:
            df_1.at[i,'Order']=np.log(df_1.at[i,'Error']/df_1.at[i-1,'Error'])/np.log(2)  # problème rencontré avec e_k+1 

    for i in range(1,df_2.shape[0]+1):

        if i>1:

            df_2.at[i,'Order']=np.log(df_2.at[i,'Error']/df_2.at[i-1,'Error'])/np.log(2)  



    display('Results for the '+ schema_choix + ' Scheme ',inter[0],df_1,inter[1],df_2)
    #return [df_1,df_2] #remplacer display par ça si on veut la conversion en fichier xlsx [cf. plus bas]



## Fuction qui calcule tout les résultats des tabelaux à remplir 

### Function pour afficher les résultats

In [48]:
def affiche_all_tab():
    choix=['EE','IE','CN']
    for i in choix:
        print('################')
        mother_of_all_func(i)

In [50]:
#affiche_all_tab()         #run cette ligne pour afficher. 

### Résulats convertis en ficher xlsx 

In [51]:
#run seulement après avoir changer le display en return dans mother_of_all_func()

def save_results_xlsx():
    choix=['EE','IE','CN']
    with pd.ExcelWriter('Resultats_TP.xlsx') as writer: 
        for i in choix:
            d=mother_of_all_func(i)
            tab1=d[0]
            tab2=d[1]

            tab1.to_excel(writer, sheet_name=i+' 1')
            tab2.to_excel(writer, sheet_name=i+' 2')

#le fichier xlsx se trouve dans le répertoire où vous avez ouvert le Jupyter Notebook 

In [ ]:
#save_results_xlsx()             #run seulement après avoir changer le display en return dans mother_of_all_func()

# Rapport et Analyse de Résultats 


### Stabilité et Consistance

Dans le schéma (EE), il est est établie que la condition CFL est vérifiée par conséquent le schéma est
monotone et stable.

Afin de mieux comprendre l'origine des oscillations, on étudie la matrice d'amplification B. Pour I=N=10, on constate que les coefficients de B sont presque tous positifs et inférieurs à 1, à l'exception d'un unique coefficient qui est négatif.


De plus, on observe que ||B||∞=0.994 et ||B||2=0.987, tandis que la CFL est de 0.484, ce qui est inférieur à 1 et confirme la stabilité monotone du schéma.

In [ ]:
#run le code suivant

B=final_func(10,10,'EE')
norm_ampl_matrix_CLF(B[2],B[3])


Lorsque N=10 et I=50, les coefficients de B ne sont pas tous positifs, et ||B||∞ est de 18.218 tandis que ||B||2 est de 16.47397, des valeurs très supérieures à 1. La CFL pour cette configuration est de 10.404, ce qui n'est pas surprenant car cette valeur est largement supérieure à 1, confirmant ainsi que la condition CFL n'est pas respectée. Par conséquent, le schéma EE avec N=10 et I=50 n'est ni monotone ni stable.

In [ ]:
#run le code suivant
B=final_func(50,10,'EE')
norm_ampl_matrix_CLF(B[2],B[3])


### Analyse des Résultats (tableaux)

#### Euler Explicit  : EE      

On constate que les résultats sont satisfaisants pour I=N jusqu'à N=80.
En effet, cela est dû au fait que la condition CFL n'est plus satisfaite, ce qui rend le schéma instable.On remarque également que l'odre prend de très grande valeur après I=80 ce qui démontre l'instablitié du schéma. 

condition CFL est satisfaite pour N=I^2/10. Le schéma est, comme on peut le constater stable.
l'ordre reste assez faible et stable comparé à I=N. l’erreur tend vers 0. On remarque également un temps de calcul un peu plus élevé notament à cause du maillage qui s'affine. 

In [30]:
#run le code suivant 

mother_of_all_func('EE')

'Results for the EE Scheme '

'I=N'

,I,N,U_barre,Error,Order
1,10.0,10.0,7.385319e+00,4.740242e-02,NaN
2,20.0,20.0,7.578832e+00,1.461111e-01,1.624033
3,40.0,40.0,7.453731e+00,2.100970e-02,-2.797938
4,80.0,80.0,2.830769e+08,2.830769e+08,33.649419
5,160.0,160.0,1.212330e+78,1.212330e+78,231.311556


'N=(I**2)/10'

,I,N,U_barre,Error,Order
1,10.0,10.0,7.385319,0.047402,NaN
2,20.0,40.0,7.624265,0.191543,2.014638
3,40.0,160.0,7.483487,0.050766,-1.915728
4,80.0,640.0,7.445799,0.013078,-1.956706
5,160.0,2560.0,7.436041,0.003320,-1.978056


#### Euler Implicit IE

Lorsque la taille de pas I est égale à N dans le schéma implicite d'Euler, la méthode numérique est considérée comme conditionnellement stable. Dans ce cas, la stabilité de la méthode dépend de la relation entre le paramètre alpha et la taille du pas de temps delta. Si cette condition est vérifiée, on peut alors calculer η et trouver une valeur bien définie et bornée.

Pour N égal à I au carré, L'erreur reste de l'ordre proche de 1, ce qui signifie que la méthode reste stable pour une large plage de valeurs de I et que la stabilité n'est pas significativement affectée par les variations de la taille du pas de temps du prix de l'actif. En outre, la méthode est monotone, ce qui implique que le prix de l'option à chaque pas de temps ne diminue pas lorsque le prix de l'actif augmente, une caractéristique souhaitable dans les applications financières.

In [52]:
#run le code suivant 

mother_of_all_func('IE')

'Results for the IE Scheme '

'I=N'

,I,N,U_barre,Error,Order
1,10.0,10.0,7.453638,0.020917,NaN
2,20.0,20.0,7.602018,0.169297,3.016807
3,40.0,40.0,7.460811,0.028090,-2.591421
4,80.0,80.0,7.432033,0.000688,-5.352101
5,160.0,160.0,7.428570,0.004151,2.593733
6,320.0,320.0,7.429675,0.003047,-0.446391


'N=I/10'

,I,N,U_barre,Error,Order
1,10.0,1.0,6.379921,1.052800,NaN
2,20.0,2.0,7.035495,0.397226,-1.406198
3,40.0,4.0,7.179283,0.253438,-0.648328
4,80.0,8.0,7.289751,0.142970,-0.825922
5,160.0,16.0,7.356802,0.075919,-0.913175
6,320.0,32.0,7.393601,0.039120,-0.956562


#### Crank-Nicolson CN

On constate que le schéma de Crank_nicolson est assez stable pour I=N. L'odre est égaelment inférieur à la racine carré de delta t. La même remarquqe est applicable à N=I/10

In [62]:
#run le code suivant 

mother_of_all_func('CN')

'Results for the CN Scheme '

'I=N'

,I,N,U_barre,Error,Order
1,10.0,10.0,7.419205,0.013516,NaN
2,20.0,20.0,7.590105,0.157383,3.541546
3,40.0,40.0,7.457187,0.024466,-2.685450
4,80.0,80.0,7.430539,0.002182,-3.487155
5,160.0,160.0,7.427866,0.004855,1.153993
6,320.0,320.0,7.429329,0.003392,-0.517235


'N=I/10'

,I,N,U_barre,Error,Order
1,10.0,1.0,6.058045,1.374677,NaN
2,20.0,2.0,6.908417,0.524304,-1.390617
3,40.0,4.0,7.138036,0.294685,-0.831229
4,80.0,8.0,7.272781,0.159940,-0.881646
5,160.0,16.0,7.349323,0.083398,-0.939448
6,320.0,32.0,7.390026,0.042695,-0.965937


### Temps de calcul 

En apportant une légère modification au code qui consiste à utiliser la méthode spsolve pour calculer la matrice Q, nous constatons une nette amélioration de la vitesse d'exécution par rapport à la méthode précédente. On pose  I=400 et N=45, en utilisant le schéma Euler Implicit. Le temps d'exécution obtenu avec la méthode initiale est de 0.1456724s. En revanche, le temps d'exécution obtenu avec la méthode spsolve est de 0.07634892s.

In [64]:
affiche_temps=final_func(400,45,'IE')
#à comparer en utilisant les fcts de création de Q avec spsolve. 
print(affiche_temps[4])

0.12172794342041016
